In [12]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

filename = "clean_ukr.txt"

pairs = []
with open(filename, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        if "\t" not in line:
            continue

        en, uk = line.split("\t", maxsplit=1)

        pairs.append((en, uk))

print("Rows amount:", len(pairs))
pairs[:5]

Rows amount: 160049


[('Go.', 'Йди.'),
 ('Hi.', 'Вітаю!'),
 ('Hi.', 'Привіт.'),
 ('Hi.', 'Привіт!'),
 ('Run!', 'Біжіть!')]

In [5]:
input_texts = [p[0] for p in pairs]
target_texts = ["<start> " + p[1] + " <end>" for p in pairs]
tokenizer_en = Tokenizer(filters='')
tokenizer_en.fit_on_texts(input_texts)
input_sequences = tokenizer_en.texts_to_sequences(input_texts)
tokenizer_uk = Tokenizer(filters='')
tokenizer_uk.fit_on_texts(target_texts)
target_sequences = tokenizer_uk.texts_to_sequences(target_texts)
num_encoder_tokens = len(tokenizer_en.word_index) + 1
num_decoder_tokens = len(tokenizer_uk.word_index) + 1
num_encoder_tokens, num_decoder_tokens

(18948, 48559)

In [6]:
max_encoder_len = max(len(seq) for seq in input_sequences)
max_decoder_len = max(len(seq) for seq in target_sequences)

encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_len, padding='post')
decoder_input_data = pad_sequences(target_sequences, maxlen=max_decoder_len, padding='post')

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
decoder_target_data[:, -1] = 0

In [7]:
latent_dim = 256

encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(num_decoder_tokens, latent_dim)
decoder_embedding = dec_emb(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │  4,850,688 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 256) │ 12,431,104 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    525,312 │ embedding_1[0][0… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │ 12,479,663 │ lstm_1[0][0]      │
│                     │ 48559)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 30,812,079 (117.54 MB)

 Trainable params: 30,812,079 (117.54 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.fit(
    [encoder_input_data, decoder_input_data],
    np.expand_dims(decoder_target_data, -1),
    batch_size=64,
    epochs=5,
)

Epoch 1/5
2501/2501 ━━━━━━━━━━━━━━━━━━━━ 389s 156ms/step - loss: 0.8898
Epoch 2/5
2501/2501 ━━━━━━━━━━━━━━━━━━━━ 391s 156ms/step - loss: 0.6595
Epoch 3/5
2501/2501 ━━━━━━━━━━━━━━━━━━━━ 390s 156ms/step - loss: 0.4891
Epoch 4/5
2501/2501 ━━━━━━━━━━━━━━━━━━━━ 390s 156ms/step - loss: 0.3660
Epoch 5/5
2501/2501 ━━━━━━━━━━━━━━━━━━━━ 391s 156ms/step - loss: 0.2777


In [10]:
encoder_model_inf = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb2 = dec_emb(decoder_inputs)
decoder_outputs2, h2, c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_states2 = [h2, c2]

decoder_model_inf = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

reverse_uk = {i: w for w, i in tokenizer_uk.word_index.items()}

In [15]:
def translate_sentence(sentence):
    seq = tokenizer_en.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_encoder_len, padding='post')

    states = encoder_model_inf.predict(seq)

    target_seq = np.array([[tokenizer_uk.word_index['<start>']]])

    stop = False
    translated = ""

    while not stop:
        output_tokens, h, c = decoder_model_inf.predict([target_seq] + states)

        token_id = np.argmax(output_tokens[0, -1, :])

        if token_id == tokenizer_uk.word_index['<end>']:
            break

        word = reverse_uk.get(token_id, "")
        translated += word + " "

        target_seq = np.array([[token_id]])
        states = [h, c]

        if len(translated.split()) > max_decoder_len:
            break

    return translated.strip()

In [19]:
print(translate_sentence("She enjoys working."))
print(translate_sentence("She forgave him."))
print(translate_sentence("She is Canadian."))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
вона любить працювати.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
вона його пробачила.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
вона канадієць.
